In [1]:
%load_ext tensorboard
!pip install matplotlib
!pip install scipy

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 34.5 MB 1.7 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os


import keras
from keras import Model
import keras.layers as layers
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.callbacks import TensorBoard, EarlyStopping
from keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator

from keras import datasets
from keras.utils import to_categorical

from __future__ import absolute_import, division, print_function, unicode_literals

In [3]:
img_train_path = os.path.join("DATASET","TRAIN")
img_test_path = os.path.join("DATASET","TEST")
data_gen = ImageDataGenerator(rescale=1./255, fill_mode='reflect', validation_split=0.33)
test_gen = ImageDataGenerator(rescale=1./255)
BATCH_SIZE = 128

train_generator = data_gen.flow_from_directory(img_train_path, target_size=(256,256), batch_size=BATCH_SIZE, subset='training')
validation_generator = data_gen.flow_from_directory(img_train_path, target_size=(256,256), batch_size=BATCH_SIZE, subset='validation')
test_generator = test_gen.flow_from_directory(img_test_path, target_size=(256,256), batch_size=BATCH_SIZE)

Found 15119 images belonging to 2 classes.
Found 7445 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


In [4]:
img, _= next(train_generator)
print(img.shape)

(128, 256, 256, 3)


In [5]:
model = keras.Sequential()

model.add(Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(256,256,3), padding = 'Same'))
model.add(MaxPooling2D())

model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu', padding = 'Same'))
model.add(MaxPooling2D())





model.add(Flatten())

model.add(Dense(units=120, activation='relu'))

model.add(Dense(units=84, activation='relu'))

model.add(Dense(units=2, activation = 'sigmoid'))

2023-03-14 06:18:47.331072: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7384 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:83:00.0, compute capability: 6.1


In [6]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.SGD(), metrics=['accuracy'], jit_compile=True)
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
callbacks_tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)
# reduce time and overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 6)       456       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 6)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 16)      2416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 16)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 65536)             0         
                                                                 
 dense (Dense)               (None, 120)               7

In [8]:
steps_per_epoch = train_generator.labels.shape[0]//BATCH_SIZE
validation_steps = validation_generator.labels.shape[0]//BATCH_SIZE

model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, 
                    validation_data=validation_generator, validation_steps=validation_steps, 
                    shuffle=True, callbacks=[callbacks_tensorboard, early_stopping])

Epoch 1/10


2023-03-14 06:18:49.528698: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x1a6aba90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-14 06:18:49.528772: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2023-03-14 06:18:49.560765: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-14 06:18:50.183777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-14 06:18:52.359822: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:5340] Disabling cuDNN frontend for the following convolution:
  input: {count: 128 feature_map_count: 3 spatial: 256 256  value_min: 0.000000 value_max: 0.000000 layout: BatchDepthYX}
  filter: {output_feature_map_count: 6 input_feature_map_count: 3 layout: OutputIn

  1/118 [..............................] - ETA: 10:32 - loss: 0.7561 - accuracy: 0.4219

2023-03-14 06:18:53.703304: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


105/118 [=========================>....] - ETA: 3s - loss: 0.5501 - accuracy: 0.7269

...

2023-03-14 06:23:00.333361: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1887436800 exceeds 10% of free system memory.


118/118 [==============================] - 66s 561ms/step - loss: 0.4492 - accuracy: 0.8026 - val_loss: 0.4501 - val_accuracy: 0.7826
Epoch 5/10
118/118 [==============================] - ETA: 0s - loss: 0.4315 - accuracy: 0.8100

2023-03-14 06:24:06.329192: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1887436800 exceeds 10% of free system memory.


118/118 [==============================] - 66s 557ms/step - loss: 0.4315 - accuracy: 0.8100 - val_loss: 0.4757 - val_accuracy: 0.7764
Epoch 6/10
118/118 [==============================] - ETA: 0s - loss: 0.4192 - accuracy: 0.8134

KeyboardInterrupt: 

In [ ]:
img, _= next(validation_generator)
plt.figure(figsize=(15, 13))
result = model.predict(img)
for i in range(30):
    ax = plt.subplot(6, 6, i + 1)
    plt.imshow(img[i])
    if result[i][1] < 0.5:
        pred = 0
        if pred == _[i][1]:
            plt.title("Organic", color='green')
        else: 
            plt.title("Organic", color='red')
    else:
        pred = 1
        if pred == _[i][1]:
            plt.title("Non-Organic", color='green')
        else: 
            plt.title("Non-Organic", color='red')
    plt.axis("off")

In [ ]:
results = model.evaluate(test_generator)
print("test loss, test acc:", results)

In [ ]:
# %tensorboard --logdir logs/fit